# UseCase Manager - Technical Walkthrough

This notebook demonstrates the core components of UseCase Manager and how they connect together.

## Architecture
```
User Interface (NiceGUI app.py)
         |
   Agent (Natural language → Tool calls)
         |
   Service Layer (Business logic + Permissions)
         |
   Models (SQLAlchemy)
         |
   Database (SQLite)
```

Explaination from bottom up.

---

## 1. Database Layer - The Foundation

The data model consists of 5 main entities:
- **UseCase** - The main entity (tracks AI/ML projects)
- **Company** - Organizations working on use cases
- **Industry** - Sectors (Energy, Manufacturing, Healthcare, etc.)
- **Person** - Contributors to use cases
- **User** - System users with roles (Reader, Maintainer, Admin)

In [ ]:
import sys
sys.path.append('.')


from models import UseCase, Company, Industry, Person, User
from models.base import SessionLocal, engine, Base

In [ ]:
# Create database session
db = SessionLocal()

Create Idustry, Company, and Person

In [ ]:
print("1: Create an Industry")

# Create Automotive industry
automotive = Industry(name="Automotive")
db.add(automotive)
db.commit()
db.refresh(automotive)  

print(f"\nCreated Industry: {automotive.name} (ID: {automotive.id})")

In [ ]:
print("2: Create Companies in that Industry")
# Create BMW
bmw = Company(
    name="BMW",
    industry_id=automotive.id
)
db.add(bmw)

# Create Tesla
tesla = Company(
    name="Tesla",
    industry_id=automotive.id
)
db.add(tesla)

db.commit()
db.refresh(bmw)
db.refresh(tesla)
print(f"\nCreated Company: {bmw.name} (ID: {bmw.id}) in {bmw.industry.name}")
print(f"Created Company: {tesla.name} (ID: {tesla.id}) in {tesla.industry.name}")

In [ ]:
print("3: Create Persons at these Companies")

# Create persons at BMW
dr_mueller = Person(
    name="Dr. Stefan Müller",
    role="Head of AI Research",
    company_id=bmw.id
)
db.add(dr_mueller)

anna_schmidt = Person(
    name="Anna Schmidt",
    role="Product Manager",
    company_id=bmw.id
)
db.add(anna_schmidt)

# Create person at Tesla
john = Person(
    name="John Smith",
    role="AI Lead",
    company_id=tesla.id
)
db.add(john)

db.commit()
db.refresh(dr_mueller)
db.refresh(anna_schmidt)
db.refresh(john)

print(f"\nCreated Person: {dr_mueller.name} ({dr_mueller.role})")
print(f"Company: {dr_mueller.company.name}")

print(f"\nCreated Person: {anna_schmidt.name} ({anna_schmidt.role})")
print(f"Company: {anna_schmidt.company.name}")

print(f"\nCreated Person: {john.name} ({john.role})")
print(f"Company: {john.company.name}")


Create use case and connect information

In [ ]:
print("4: Create Use Cases")

# Create a use case at BMW
autonomous_driving = UseCase(
    title="Autonomous Driving AI",
    description="Development of Level 4 autonomous driving using computer vision, "
                "sensor fusion, and deep learning for highway and urban scenarios.",
    expected_benefit="Reduce accidents by 40%, improve traffic flow, enable new mobility services",
    status="in_progress",
    company_id=bmw.id,
    industry_id=automotive.id
)
db.add(autonomous_driving)

# Create a use case at Tesla
battery_ai = UseCase(
    title="Battery Management AI",
    description="AI system to optimize battery charging, cooling, and predict degradation patterns",
    expected_benefit="Extend battery life by 15%, reduce charging time by 20%",
    status="approved",
    company_id=tesla.id,
    industry_id=automotive.id
)
db.add(battery_ai)

db.commit()
db.refresh(autonomous_driving)
db.refresh(battery_ai)

print(f"\nCreated Use Case: {autonomous_driving.title}")
print(f"   Company: {autonomous_driving.company.name}")
print(f"   Industry: {autonomous_driving.industry.name}")
print(f"   Status: {autonomous_driving.status}")

print(f"\nCreated Use Case: {battery_ai.title}")
print(f"   Company: {battery_ai.company.name}")
print(f"   Industry: {battery_ai.industry.name}")
print(f"   Status: {battery_ai.status}")

In [ ]:
# Link Persons to Use Cases
print("5: Assign Contributors to Use Cases")

# Add contributors to autonomous driving project
autonomous_driving.persons.append(dr_mueller)
autonomous_driving.persons.append(anna_schmidt)

# Add contributor to battery AI project
battery_ai.persons.append(john)

db.commit()

print(f"\nAssigned contributors to '{autonomous_driving.title}':")
for person in autonomous_driving.persons:
    print(f"{person.name} ({person.role})")

print(f"\nAssigned contributors to '{battery_ai.title}':")
for person in battery_ai.persons:
    print(f"{person.name} ({person.role})")


# Query database and its information

In [ ]:
print("QUERYING THE DATA CREATED")

# Query all use cases
all_use_cases = db.query(UseCase).all()

print(f"\nTotal Use Cases: {len(all_use_cases)}")

for uc in all_use_cases:
    print(f"\n{uc.title}")
    print(f"   Company: {uc.company.name}")
    print(f"   Industry: {uc.industry.name}")
    print(f"   Status: {uc.status}")
    print(f"   Contributors: {', '.join([p.name for p in uc.persons])}")

In [ ]:
# Cell 10: Query by relationships
print("QUERY BY RELATIONSHIPS")

# Find all use cases in Automotive
automotive_use_cases = db.query(UseCase).filter(
    UseCase.industry_id == automotive.id
).all()

print(f"\nUse Cases in {automotive.name} Industry: {len(automotive_use_cases)}")
for uc in automotive_use_cases:
    print(f"   {uc.title} ({uc.company.name})")

# Find all use cases at BMW
bmw_use_cases = db.query(UseCase).filter(
    UseCase.company_id == bmw.id
).all()

print(f"\nUse Cases at {bmw.name}: {len(bmw_use_cases)}")
for uc in bmw_use_cases:
    print(f"   {uc.title}")

# Find all use cases Dr. Müller contributes to
mueller_use_cases = db.query(UseCase).join(
    UseCase.persons
).filter(
    Person.id == dr_mueller.id
).all()

print(f"\nUse Cases with {dr_mueller.name} as contributor: {len(mueller_use_cases)}")
for uc in mueller_use_cases:
    print(f"   {uc.title}")

# Service Layer

In [ ]:
from services import UseCaseService
from services.user_service import UserService

In [ ]:
# Initialize services
user_service = UserService()
service = UseCaseService()
# For now, authenticate as admin - roles shown later
admin = user_service.authenticate("admin@example.com", "admin123")

# Creating idustry, Company, Person and use case via service layer

In [ ]:
print("CREATING INDUSTRY VIA SERVICE LAYER")

# Create Aerospace industry
industry = service.create_industry("Aerospace", current_user=admin)

print(f"\nCreated Industry: {industry['name']} (ID: {industry['id']})")

In [ ]:
print("CREATING COMPANY VIA SERVICE LAYER")

# Create SpaceX
company = service.create_company(
    "SpaceX", 
    industry['id'], 
    current_user=admin
)

print(f"\nCreated Company: {company['name']} (ID: {company['id']})")
print(f"   Industry: {company['industry_name']}")

In [ ]:
print("CREATING PERSON VIA SERVICE LAYER")

# Create engineer at SpaceX
person = service.create_person(
    name="Maria Garcia",
    role="Lead Engineer",
    company_id=company['id'],
    current_user=admin
)

print(f"\nCreated Person: {person['name']}")
print(f"   Role: {person['role']}")
print(f"   Company: {person['company_name']}")

print("\nReturned dictionary contains:")
for key in person.keys():
    print(f"   {key}: {person[key]}")

In [ ]:
print("CREATING USE CASE VIA SERVICE LAYER")

# Create a use case
use_case = service.create_use_case(
    title="Rocket Landing",
    company_id=company['id'],
    industry_id=industry['id'],
    description="AI system for autonomous rocket landing using computer vision, "
                "trajectory optimization, and real-time control",
    expected_benefit="Achieve 99 percent landing success rate, reduce costs by reusing rockets",
    status="in_progress",
    current_user=admin
)

print(f"\n Created Use Case: {use_case['title']}")
print(f"   ID: {use_case['id']}")
print(f"   Company: {use_case['company_name']}")
print(f"   Industry: {use_case['industry_name']}")
print(f"   Status: {use_case['status']}")

print("\n Use case details:")
print(f"   Description: {use_case['description'][:80]}...")
print(f"   Expected Benefit: {use_case['expected_benefit'][:80]}...")

# Query via service layer

In [ ]:
print("QUERYING VIA SERVICE LAYER")

# Get all use cases
all_use_cases = service.get_all_use_cases(current_user=admin)
print(f"\n Total use cases in database: {len(all_use_cases)}")

# Show first few
print("\nAll use cases:")
for uc in all_use_cases: 
    print(f"   {uc['title']} ({uc['company_name']}) - {uc['status']}")

# Filter by industry
print(f"\nFiltering by Industry: {industry['name']}")
aerospace_ucs = service.filter_use_cases(
    industry_id=industry['id'],
    current_user=admin
)
print(f"   Found {len(aerospace_ucs)} use cases in Aerospace")
for uc in aerospace_ucs:
    print(f"   • {uc['title']}")

# Filter by status
print(f"\nFiltering by Status: 'in_progress'")
in_progress_ucs = service.filter_use_cases(
    status="in_progress",
    current_user=admin
)
print(f"   Found {len(in_progress_ucs)} use case(s) in progress")
for uc in in_progress_ucs:
    print(f"   {uc['title']} ({uc['company_name']})")

# Update use case

In [ ]:
print("UPDATE OPERATIONS VIA SERVICE LAYER")

# Update use case details
print("\nUpdating use case title and description...")
updated = service.update_use_case(
    use_case_id=use_case['id'],
    title="Rocket Landing v2.0",
    description="Enhanced AI system with improved trajectory prediction and weather adaptation",
    current_user=admin
)

print(f"Updated title: {updated['title']}")
print(f"Updated description: {updated['description'][:60]}...")

# Change status
print("\nChanging status from 'in_progress' to 'approved'...")
status_updated = service.update_use_case_status(
    use_case['id'],
    "approved",
    current_user=admin
)

print(f"New status: {status_updated['status']}")

# Link person to use cas

In [ ]:
print("Linking Persons to Use Cases")

# Add person as contributor to use case
print(f"\nAdding {person['name']} as contributor to '{use_case['title']}'")

service.add_persons_to_use_case(
    use_case['id'],
    [person['id']],  
    current_user=admin
)

print(f"Added contributor")

# Verify by getting contributors
contributors = service.get_persons_by_use_case(
    use_case['id'],
    current_user=admin
)

print(f"\nContributors to '{use_case['title']}':")
for contrib in contributors:
    print(f"   {contrib['name']} ({contrib['role']}) - {contrib['company_name']}")

# get use case

In [ ]:
print("RETRIEVING FULL USE CASE DETAILS")


# Get use case by ID (includes all related data)
full_details = service.get_use_case_by_id(use_case['id'], current_user=admin)

print(f"\nFull Details for Use Case #{full_details['id']}:")
print(f"\n   Title: {full_details['title']}")
print(f"   Status: {full_details['status']}")
print(f"   Company: {full_details['company_name']}")
print(f"   Industry: {full_details['industry_name']}")
print(f"\n   Description:")
print(f"   {full_details['description']}")
print(f"\n   Expected Benefit:")
print(f"   {full_details['expected_benefit']}")

# Agent

### How the Agent Works
```
User Query (Natural Language)
         ↓
   Claude Sonnet 3.5 (LLM)
         ↓
   Decides which tool(s) to call
         ↓
   Tool Executor (with permission checks)
         ↓
   Service Layer
         ↓
   Database
         ↓
   Results back to LLM
         ↓
   Natural Language Response
```

---

### Multi-Round Tool Calling

The agent can call **multiple tools in sequence** to accomplish complex tasks:

**Example: "Create a use case for Mercedes in Automotive"**

**Round 1:**
- Tool: `get_all_companies()` → Check if Mercedes exists
- Tool: `get_all_industries()` → Check if Automotive exists

**Round 2:**
- Tool: `create_company("Mercedes", automotive_id)` → Create if missing
- Tool: `create_industry("Automotive")` → Create if missing

**Round 3:**
- Tool: `create_use_case(...)` → Create the use case

**Response:**
- "I've successfully created the use case..."

---

In [ ]:
from agent import run_agent
from agent.tool_executor import set_current_user

set_current_user(admin)

## Examples

In [ ]:
print("1: Simple Query")

query = "Show me all use cases in the Aerospace industry"

print(f"\nUser: {query}")
print("\n" + "-"*70)

response = run_agent(
    query,
    verbose=True,
    max_rounds=10
)

print("\n" + "-"*70)
print(f"\nAgent Response:\n{response}")

In [ ]:
print("Example 2: Multi-Step Creation")

query = """Create a use case for Mercedes called 'Predictive Quality Control AI' 
in the Automotive industry. Description: AI vision system for defect detection. 
Expected benefit: Reduce defects by 30%."""

print(f"\nUser: {query}")
print("\n" + "-"*70)

response = run_agent(
    query,
    verbose=True,
    max_rounds=10
)

print("\n" + "-"*70)
print(f"\nAgent Response:\n{response}")

In [ ]:
print("Example 3: Update via Natural Language")

# First use case
all_ucs = service.get_all_use_cases(current_user=admin)
uc_to_update = all_ucs[0]  

query = f"Change the status of use case {uc_to_update['id']} to completed"

print(f"\nUser: {query}")
print(f"({uc_to_update['title']}')")
print("\n" + "-"*70)

response = run_agent(
    query,
    verbose=True,
    max_rounds=10
)

print("\n" + "-"*70)
print(f"\nAgent Response:\n{response}")

# Verify the change
updated_uc = service.get_use_case_by_id(uc_to_update['id'], current_user=admin)
print(f"\nVerified: Status is now '{updated_uc['status']}'")

In [ ]:
print("\n" + "="*70)
print("Example 4: Contextual Query")
print("="*70)

query = "Which use cases are in progress?"

print(f"\nUser: {query}")
print("\n" + "-"*70)

response = run_agent(
    query,
    verbose=True,
    max_rounds=10
)

print("\n" + "-"*70)
print(f"\nAgent Response:\n{response}")

In [ ]:
print("Example 5: Multiple Filters")

query = "Show me approved use cases in the Automotive industry"

print(f"\nUser: {query}")
print("\n" + "-"*70)

response = run_agent(
    query,
    verbose=True,
    max_rounds=10
)

print("\n" + "-"*70)
print(f"\nAgent Response:\n{response}")


In [ ]:
print("Example 6: Managing Contributors")

# Get a use case and person from the same company
aerospace_uc = service.filter_use_cases(industry_id=industry['id'], current_user=admin)[0]
spacex_persons = [p for p in service.get_all_persons(current_user=admin) 
                  if p['company_name'] == 'SpaceX']

print(aerospace_uc['title'], spacex_persons)

if spacex_persons:
    query = f"Add {spacex_persons[0]['name']} as a contributor to use case {aerospace_uc['id']}"
    
    print(f"\nUser: {query}")
    print("\n" + "-"*70)
    
    response = run_agent(
        query,
        verbose=True,
        max_rounds=10
    )
    
    print("\n" + "-"*70)
    print(f"\nAgent Response:\n{response}")
    
    # Verify
    contributors = service.get_persons_by_use_case(aerospace_uc['id'], current_user=admin)
    print(f"\nVerified contributors:")
    for c in contributors:
        print(f"   {c['name']} ({c['role']})")

# Transcript Extraction

**INPUT: Raw workshop transcript**


**Step 1: LLM Extraction**
- Claude reads transcript
- Identifies use cases
- Extracts: company, title, description, benefits, people
- Outputs: Structured prompts for agent to create use case


**Step 2: Agent Creation**
- For each prompt: Agent does his thing as already shown above

In [ ]:
print("LOADING WORKSHOP TRANSCRIPT")

# Load one of our test transcripts
with open('test_data/transcripts/energy_workshop_transcript.txt', 'r', encoding='utf-8') as f:
    transcript = f.read()

print(f"\nLoaded transcript: {len(transcript)} characters")

print("\nStarts with:")
print("-"*70)
print(transcript[:800])
print("...")
print("-"*70)

In [ ]:
# Cell 34: Extract Use Case Prompts
from extraction.transcript_processor import extract_prompts_from_transcript

print("Step 1: Extracting Use Cases with LLM")

print("\nSending transcript to Claude Sonnet 3.5...")
print("Asking it to identify use cases and extract:")
print("   Company/Organization")
print("   Use case title")
print("   Description")
print("   Expected benefits")
print("   People involved")

print("\n" + "-"*70)

# Extract prompts (this calls the LLM)
prompts = extract_prompts_from_transcript(transcript, verbose=True)

print("-"*70)

print(f"\nExtraction complete!")
print(f"Found {len(prompts)} use case(s) in the transcript")

In [ ]:
print("EXTRACTED USE CASE PROMPTS")

for i, prompt in enumerate(prompts, 1):
    print(f"\nUse Case {i}:")
    print("-"*70)
    print(prompt)
    print("-"*70)

# Use agent to create use cases

In [ ]:
print("Step 2: Creating Use Cases via Agent")

set_current_user(admin)

created_use_cases = []

for i, prompt in enumerate(prompts, 1):
    print(f"\n{'='*70}")
    print(f"Creating Use Case {i}/{len(prompts)}")
    print('='*70)
    
    print(f"\nPrompt to agent:")
    print(f"{prompt[:150]}...")
    
    print("\n" + "-"*70)
    
    # Call agent with the prompt
    response = run_agent(
        prompt,
        verbose=True,
        max_rounds=10
    )
    
    print("-"*70)
    
    print(f"\nAgent Response:")
    print(response[:200] + "..." if len(response) > 200 else response)
    
    created_use_cases.append(response)



In [ ]:
print("Check Database")  # to verify

# Get all use cases from Energy industry
energy_industry = [ind for ind in service.get_all_industries(current_user=admin) 
                   if ind['name'] == 'Energy']

if energy_industry:
    energy_ucs = service.filter_use_cases(
        industry_id=energy_industry[0]['id'],
        current_user=admin
    )
    
    print(f"\nUse Cases in Energy Industry: {len(energy_ucs)}")
    print("\nCreated from transcript:")
    
    for uc in energy_ucs[-len(prompts):]:  # Show the newest ones
        print(f"\n{uc['title']}")
        print(f"   Company: {uc['company_name']}")
        print(f"   Status: {uc['status']}")
        
        # Get contributors
        contributors = service.get_persons_by_use_case(uc['id'], current_user=admin)
        if contributors:
            print(f"   Contributors:")
            for person in contributors:
                print(f"      • {person['name']} ({person['role']})")
else:
    print("\nError")

# Permissions


So far, everything done with admin rights.

UseCaseManager has **three roles**:

| Role | Capabilities |
|------|-------------|
| **Reader** | View use cases, chat for read queries only |
| **Maintainer** | Reader + Create/Edit use cases, upload transcripts |
| **Admin** | Maintainer + Delete use cases, manage users |

**Design Decision:** Permissions are enforced at **two layers**:
1. **Service Layer** - Backend protection (every method checks permissions)
2. **UI Layer** - Frontend adapts (hides unavailable features)

In [ ]:
from services.user_service import UserService

user_service = UserService()

# Authenticate all three roles
reader = user_service.authenticate("reader@example.com", "reader123")
maintainer = user_service.authenticate("maintainer@example.com", "maintainer123")
admin = user_service.authenticate("admin@example.com", "admin123")

print("="*70)
print("TEST USERS")
print("="*70)

print(f"\nReader:     {reader['name']} ({reader['email']})")
print(f"   Role: {reader['role']}")

print(f"\nMaintainer: {maintainer['name']} ({maintainer['email']})")
print(f"   Role: {maintainer['role']}")

print(f"\nAdmin:      {admin['name']} ({admin['email']})")
print(f"   Role: {admin['role']}")


# Reading data (all should be able)

In [ ]:
print("TEST 1: Reading Data - All roles should succeed")

service = UseCaseService()

# Reader
print("\nREADER attempting to view use cases:")
try:
    reader_ucs = service.get_all_use_cases(current_user=reader)
    print(f"   SUCCESS: Can view {len(reader_ucs)} use case(s)")
except Exception as e:
    print(f"   FAILED: {e}")

# Maintainer
print("\nMAINTAINER attempting to view use cases:")
try:
    maintainer_ucs = service.get_all_use_cases(current_user=maintainer)
    print(f"   SUCCESS: Can view {len(maintainer_ucs)} use case(s)")
except Exception as e:
    print(f"   FAILED: {e}")

# Admin
print("\nADMIN attempting to view use cases:")
try:
    admin_ucs = service.get_all_use_cases(current_user=admin)
    print(f"   SUCCESS: Can view {len(admin_ucs)} use case(s)")
except Exception as e:
    print(f"   FAILED: {e}")


# Create use case (reader should not be able)

In [ ]:
print("TEST 2: Creating Use Cases - Reader should be blocked")

# Get an industry and company for testing
industries = service.get_all_industries(current_user=admin)
companies = service.get_all_companies(current_user=admin)

if industries and companies:
    test_industry = industries[0]
    test_company = companies[0]
    
    # Reader tries to create
    print("\nREADER attempting to create use case:")
    try:
        uc = service.create_use_case(
            title="Test Reader Use Case",
            company_id=test_company['id'],
            industry_id=test_industry['id'],
            current_user=reader
        )
        print(f"   UNEXPECTED: Reader should not be able to create!")
    except Exception as e:
        print(f"   BLOCKED: {e}")
    
    # Maintainer tries to create
    print("\nMAINTAINER attempting to create use case:")
    try:
        uc = service.create_use_case(
            title="Test Maintainer Use Case",
            company_id=test_company['id'],
            industry_id=test_industry['id'],
            description="Testing maintainer permissions",
            current_user=maintainer
        )
        print(f"   SUCCESS: Created use case '{uc['title']}' (ID: {uc['id']})")
        maintainer_created_id = uc['id']
    except Exception as e:
        print(f"   FAILED: {e}")
    
    # Admin tries to create
    print("\nADMIN attempting to create use case:")
    try:
        uc = service.create_use_case(
            title="Test Admin Use Case",
            company_id=test_company['id'],
            industry_id=test_industry['id'],
            description="Testing admin permissions",
            current_user=admin
        )
        print(f"   SUCCESS: Created use case '{uc['title']}' (ID: {uc['id']})")
        admin_created_id = uc['id']
    except Exception as e:
        print(f"   FAILED: {e}")

# Updateing use case (user should not be able)

In [ ]:
print("TEST 3: Updating Use Cases - Reader should be blocked")

# Use the maintainer-created use case from previous test

print("\nREADER attempting to update use case:")
try:
    updated = service.update_use_case(
        use_case_id=maintainer_created_id,
        title="Reader Modified Title",
        current_user=reader
    )
    print(f"   UNEXPECTED: Reader should not be able to update!")
except Exception as e:
    print(f"   BLOCKED: {e}")

# Maintainer tries to update
print("\nMAINTAINER attempting to update use case:")
try:
    updated = service.update_use_case(
        use_case_id=maintainer_created_id,
        title="Maintainer Modified Title",
        current_user=maintainer
    )
    print(f"   SUCCESS: Updated to '{updated['title']}'")
except Exception as e:
    print(f"   FAILED: {e}")

# Admin tries to update
print("\nADMIN attempting to update use case:")
try:
    updated = service.update_use_case(
        use_case_id=maintainer_created_id,
        title="Admin Modified Title",
        current_user=admin
    )
    print(f"   SUCCESS: Updated to '{updated['title']}'")
except Exception as e:
    print(f"   FAILED: {e}")

# Deleting use cases (only admin shoud be enabled)

In [ ]:
print("TEST 4: Deleting Use Cases - Only Admin should succeed")


# Reader tries to delete
print("\nREADER attempting to delete use case:")
try:
    service.delete_use_case(maintainer_created_id, current_user=reader)
    print(f"   UNEXPECTED: Reader should not be able to delete!")
except Exception as e:
    print(f"   BLOCKED: {e}")

# Maintainer tries to delete
print("\nMAINTAINER attempting to delete use case:")
try:
    service.delete_use_case(maintainer_created_id, current_user=maintainer)
    print(f"   UNEXPECTED: Maintainer should not be able to delete!")
except Exception as e:
    print(f"   BLOCKED: {e}")

# Admin tries to delete
print("\nADMIN attempting to delete use case:")
try:
    deleted = service.delete_use_case(maintainer_created_id, current_user=admin)
    print(f"   SUCCESS: Deleted use case '{deleted['title']}'")
except Exception as e:
    print(f"   FAILED: {e}")

# Clean up the second test use case
try:
    service.delete_use_case(admin_created_id, current_user=admin)
except:
    pass

# Agent with different roles

In [ ]:
print("TEST 5: Agent Respects Permissions")

from agent.tool_executor import set_current_user

# Reader tries to create via agent
print("\nREADER using agent to create:")
set_current_user(reader)
print("Query: 'Create a use case called Test for BMW'")
print("-"*70)

response = run_agent(
    "Create a use case called 'Reader Test Case' for BMW in Automotive",
    verbose=True,
    max_rounds=10
)

print("-"*70)
print(f"\nAgent Response (first 200 chars):")
print(response[:200] + "...")


# Maintainer succeeds
print("\nMAINTAINER using agent to create:")
set_current_user(maintainer)
print("Query: 'Create a use case called Maintainer Test for BMW'")
print("-"*70)

response = run_agent(
    "Create a use case called 'Maintainer Test Case' for BMW in Automotive. Description: Testing permissions.",
    verbose=True,
    max_rounds=10
)

print("-"*70)
print(f"\nAgent Response (first 200 chars):")
print(response[:200] + "...")

# Clean up
try:
    # Find and delete the test case we just created
    bmw_ucs = service.filter_use_cases(company_name="BMW", current_user=admin)
    for uc in bmw_ucs:
        if "Maintainer Test Case" in uc['title']:
            service.delete_use_case(uc['id'], current_user=admin)
except:
    pass

In [ ]:
print("TEST 6: Admin-Only Operations via Agent")

# Set agent to admin
set_current_user(admin)

# First, create a test use case to delete
print("\nSetting up test use case for deletion demo...")
test_uc = service.create_use_case(
    title="Test Use Case for Deletion",
    company_id=companies[0]['id'],
    industry_id=industries[0]['id'],
    description="This use case will be deleted to demonstrate admin permissions",
    current_user=admin
)
print(f"Created test use case (ID: {test_uc['id']})")

# Create another for archiving
test_uc_archive = service.create_use_case(
    title="Test Use Case for Archiving",
    company_id=companies[0]['id'],
    industry_id=industries[0]['id'],
    description="This use case will be archived",
    status="completed",
    current_user=admin
)
print(f"Created test use case for archiving (ID: {test_uc_archive['id']})")

In [ ]:
print("ADMIN OPERATION: Archive Use Case via Agent")

query = f"Archive use case {test_uc_archive['id']}"

print(f"\nADMIN Query: '{query}'")
print(f"   (Archiving: '{test_uc_archive['title']}')")
print(f"   Current status: {test_uc_archive['status']}")
print("\n" + "-"*70)

response = run_agent(
    query,
    verbose=True,
    max_rounds=10
)

print("-"*70)
print(f"\nAgent Response:")
print(response)

# Verify archiving
archived_uc = service.get_use_case_by_id(test_uc_archive['id'], current_user=admin)
print("\nVerification:")
print(f"   Status before: {test_uc_archive['status']}")
print(f"   Status after:  {archived_uc['status']}")
if archived_uc['status'] == 'archived':
    print(f"   Successfully archived!")
else:
    print(f"   Archive may have failed")

In [ ]:
print("COMPARISON: Maintainer Attempts to Archive (Should Be Blocked)")

# Create a new test use case for this demo
print("\nCreating test use case for maintainer archive attempt...")
test_uc_maint_archive = service.create_use_case(
    title="Test Maintainer Archive Attempt",
    company_id=companies[0]['id'],
    industry_id=industries[0]['id'],
    status="completed",
    current_user=admin
)
print(f"Created (ID: {test_uc_maint_archive['id']}, Status: {test_uc_maint_archive['status']})")

# Switch to maintainer
set_current_user(maintainer)

# Maintainer tries to archive
print("\nMAINTAINER attempting to archive via agent:")
query_maint = f"Archive use case {test_uc_maint_archive['id']}"
print(f"   Query: '{query_maint}'")
print("-"*70)

response_maint = run_agent(
    query_maint,
    verbose=True,
    max_rounds=10
)

print("-"*70)
print(f"\nAgent Response:")
print(response_maint)

# Verify status unchanged
check_uc = service.get_use_case_by_id(test_uc_maint_archive['id'], current_user=admin)
print("\nVerification:")
print(f"   Status remained: {check_uc['status']}")
if check_uc['status'] != 'archived':
    print(f"   Maintainer was correctly blocked from archiving")
else:
    print(f"   Unexpected: Status was changed to archived")

# Cleanup both test use cases
print("\nCleanup:")
set_current_user(admin)
service.delete_use_case(test_uc_archive['id'], current_user=admin)
print(f"   Deleted archived test use case")
service.delete_use_case(test_uc_maint_archive['id'], current_user=admin)
print(f"   Deleted maintainer test use case")

In [ ]:
print("ADMIN OPERATION: Delete Use Case via Agent")

# Count use cases before deletion
all_ucs_before = service.get_all_use_cases(current_user=admin)
count_before = len(all_ucs_before)

query = f"Delete use case {test_uc['id']}"

print(f"\nADMIN Query: '{query}'")
print(f"   Use cases before deletion: {count_before}")
print("\n" + "-"*70)

response = run_agent(
    query,
    verbose=True,
    max_rounds=10
)

print("-"*70)
print(f"\nAgent Response:")
print(response)

# Count after deletion
all_ucs_after = service.get_all_use_cases(current_user=admin)
count_after = len(all_ucs_after)

print("\n Verification:")
print(f"   Use cases before: {count_before}")
print(f"   Use cases after:  {count_after}")
if count_after < count_before:
    print(f"   Successfully deleted ({count_before - count_after} use case removed)")
else:
    print(f"   Deletion may have failed (count unchanged)")

# History

In [ ]:
print("CONVERSATION WITH CONTEXT")

# Reset to admin
set_current_user(admin)

# Initialize conversation history
conversation_history = []

# Turn 1: Create a use case
print("─" * 70)
print("Turn 1: Create a use case")
print("─" * 70)

user_msg_1 = "Create a use case for Volkswagen called 'Assembly Line Optimization AI' in the Automotive industry. Description: AI optimizes production line efficiency. Expected benefit: Increase throughput by 20%."

print(f"\nUser: {user_msg_1}\n")

response_1 = run_agent(
    user_msg_1,
    conversation_history=None,  
    verbose=True,
    max_rounds=10
)

print(f"Agent: {response_1}\n")

# Add to history
conversation_history.append({'role': 'user', 'content': user_msg_1})
conversation_history.append({'role': 'assistant', 'content': response_1})

In [ ]:
print("Turn 2: Ask about 'it")


user_msg_2 = "What's the status of it?"

print(f"\nUser: {user_msg_2}")

response_2 = run_agent(
    user_msg_2,
    conversation_history=conversation_history,  
    verbose=True,
    max_rounds=10
)

print(f"Agent: {response_2}\n")

# Add to history
conversation_history.append({'role': 'user', 'content': user_msg_2})
conversation_history.append({'role': 'assistant', 'content': response_2})


In [ ]:
print("Turn 3: Update using context")

user_msg_3 = "Change it to in_progress status"

print(f"\nUser: {user_msg_3}")

response_3 = run_agent(
    user_msg_3,
    conversation_history=conversation_history,
    verbose=True,
    max_rounds=10
)

print(f"Agent: {response_3}\n")

# Add to history
conversation_history.append({'role': 'user', 'content': user_msg_3})
conversation_history.append({'role': 'assistant', 'content': response_3})


In [ ]:
print("Turn 4: Add contributor using context")

user_msg_4 = "Add a person named Thomas Weber as Lead Engineer at Volkswagen and make him a contributor to this use case"

print(f"\nUser: {user_msg_4}")

response_4 = run_agent(
    user_msg_4,
    conversation_history=conversation_history,
    verbose=True,  
    max_rounds=10
)

print(f"\nAgent: {response_4}\n")

# Add to history
conversation_history.append({'role': 'user', 'content': user_msg_4})
conversation_history.append({'role': 'assistant', 'content': response_4})